In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_pickle('df_1518.pkl')

In [3]:
spatial_label = pd.read_pickle('../data_processeing/spatial_labels.pkl')

In [41]:
df_new.groupby('spatial_label').count()

,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,...,Music_karaoke,Music_live,Music_no_music,Music_video,OutdoorSeating,RestaurantsDelivery,RestaurantsGoodForGroups,Smoking,review_count_greater_median,cuisine_Chinese
spatial_label,,,,,,,,,,,,,,,,,,,,,
0,892,892,892,892,892,892,892,892,892,892,...,0,175,807,807,796,792,807,792,892,892
1,1094,1094,1094,1094,1094,1094,1094,1094,1094,1094,...,0,83,932,932,687,562,932,562,1094,1094
2,5274,5274,5274,5274,5274,5274,5274,5274,5274,5274,...,1,677,4835,4835,4916,4856,4835,4856,5274,5274
3,6464,6464,6464,6464,6464,6464,6464,6464,6464,6464,...,2,595,5388,5388,5474,5366,5388,5366,6464,6464
4,952,952,952,952,952,952,952,952,952,952,...,0,125,683,683,718,639,683,639,952,952
5,2071,2071,2071,2071,2071,2071,2071,2071,2071,2071,...,0,279,1790,1790,1872,1850,1790,1850,2071,2071
6,2317,2317,2317,2317,2317,2317,2317,2317,2317,2317,...,0,161,1764,1764,1863,1808,1764,1808,2317,2317
7,3564,3564,3564,3564,3564,3564,3564,3564,3564,3564,...,3,461,3234,3234,3278,3279,3234,3279,3564,3564
8,2502,2502,2502,2502,2502,2502,2502,2502,2502,2502,...,0,404,2210,2210,2187,2171,2210,2171,2502,2502


In [4]:
df_new = pd.concat([df,spatial_label], axis=1)

In [50]:
df_select = df_new[(df_new['stars'] >= 4) & (df_new['spatial_label'] == 2) & (df_new['cuisine_Chinese'] == 2)]

# Spectual Clustering

In [51]:
X = df_select.loc[:, 'AgesAllowed': 'review_count_greater_median']
X = pd.concat([X, df_select[['stars']]], axis=1)
X['stars'] = X['stars'].apply(str)
X = pd.get_dummies(X, dummy_na=False, drop_first=True)
def true_false(x):
    if x == True:
        return 1
    else:
        return 0
X['review_count_greater_median'] = X['review_count_greater_median'].apply(true_false)

In [52]:
for n_clusters in range(2,5):  
    spectural_clustering = SpectralClustering(n_clusters=n_clusters, random_state=100, affinity='sigmoid').fit(X)
    labels = spectural_clustering.labels_
    print silhouette_score(X, labels, metric='cityblock')
    print list(labels).count(0)
    print list(labels).count(1)
    print list(labels).count(2)
    print list(labels).count(3)
    print list(labels).count(4)

0.405114162492
172
4
0
0
0
0.2347455604
170
5
1
0
0
0.227827609194
169
3
3
1
0


In [53]:
spectural_clustering = SpectralClustering(n_clusters=2, random_state=999, affinity='sigmoid').fit(X)
labels_spectural = spectural_clustering.labels_

In [54]:
df_select.shape

(176, 67)

# KMeans

In [55]:
## function that gets silhouette scores for a clustering method
def get_silhouette_score(X, cluster_method, method='complete'):
    
    #Choose a range(list) of clusters I would like to try:
    range_n_clusters = range(2,10)
    
    if cluster_method == 'kmeans':
        for n_clusters in range_n_clusters: 
            km_result = KMeans(n_clusters=n_clusters, random_state=22).fit(X) # fit model
            cluster_labels = km_result.labels_

            # clustering model & clustering result to variable "cluster_labels". 
            silhouette_avg = silhouette_score(X, cluster_labels, random_state=22)
            print("For n_clusters ={},".format(n_clusters)+"{} - average silhouette_score :{}".\
                  format(cluster_method, silhouette_avg))

        
    elif cluster_method == 'gaussian_mix':
        for n_clusters in range_n_clusters: 
            gm = GaussianMixture(n_components=n_clusters, random_state=22).fit(X) # fit model
            cluster_labels = gm.predict(X)

            # clustering model & clustering result to variable "cluster_labels". 
            silhouette_avg = silhouette_score(X, cluster_labels, random_state=22)
            print("For n_clusters ={},".format(n_clusters)+"{} - average silhouette_score :{}".\
                  format(cluster_method, silhouette_avg))
        
        # res_mix_cluster.predict(X)
        
    elif cluster_method == 'hierarchical':
        # Define Z
        Z = linkage(X, method)

        for n_clusters in range_n_clusters:   
            cluster_labels=  fcluster(Z, n_clusters, criterion='maxclust') 

            silhouette_avg = silhouette_score(X, cluster_labels, random_state=22)
            print("For n_clusters ={},".format(n_clusters)+"{} - average silhouette_score :{}".\
                  format(cluster_method, silhouette_avg))

In [56]:
## choose the number of clusters using silhouette method
get_silhouette_score(X, 'kmeans')

For n_clusters =2,kmeans - average silhouette_score :0.127166501834
For n_clusters =3,kmeans - average silhouette_score :0.125831355634
For n_clusters =4,kmeans - average silhouette_score :0.109131456769
For n_clusters =5,kmeans - average silhouette_score :0.0808942656566
For n_clusters =6,kmeans - average silhouette_score :0.0830437671919
For n_clusters =7,kmeans - average silhouette_score :0.0738487530425
For n_clusters =8,kmeans - average silhouette_score :0.084144752272
For n_clusters =9,kmeans - average silhouette_score :0.0840328521262


In [57]:
n_clusters = 2  # number of clusters
#XX= X.ix[:, ] # hour of day data

#train the model.
km=KMeans(n_clusters=n_clusters, random_state=22).fit(X)
labels_km = km.labels_

print list(labels_km).count(0)
print list(labels_km).count(1)

78
98


In [19]:
## Distance 
# KM=KMeans(n_clusters=k,random_state=9)
scor=km.fit_predict(X)

res_p=pd.DataFrame(km.transform(X))  ## distances of each data point to each cluster center!!!
res_p=pd.concat((res_p, pd.DataFrame(km.fit_predict(X))),axis=1)

res_p.columns=list(range(n_clusters))+["cluster"]
res_p.loc[:,"score"]=res_p.apply(lambda x: x[int(x["cluster"])],axis=1)
res_p.sort_values("score",ascending=False)[:5]

,0,1,cluster,score
35,3.348413,3.529255,0,3.348413
136,3.130752,3.558051,0,3.130752
64,2.945065,3.514769,0,2.945065
60,2.852182,3.348245,0,2.852182
124,3.534674,2.851160,1,2.851160


In [20]:
X_df = X.copy()

In [21]:
X_df['km'] = labels_km

In [22]:
X_df['distance_KM'] = res_p.score.values

In [23]:
## Get Anomalies
def get_anomaly(df, label_col, dist_or_likelihood, thres):
    
    df_anomalies = pd.DataFrame(columns=df.columns)
    
    clustermean = df.groupby(label_col)[dist_or_likelihood].mean()
    clusterstd = df.groupby(label_col)[dist_or_likelihood].std()
    
    for label in df[label_col].unique():
        anomaly = df[(df[label_col]==label) \
               & (abs(df[dist_or_likelihood] - clustermean[label]) / clusterstd[label] > thres)]
        
        df_anomalies = pd.concat([df_anomalies, anomaly], axis=0)
    
    return df_anomalies

In [24]:
## get anomalies based on kmeans distance
km_anomalies = get_anomaly(X_df, 'km', 'distance_KM', 2)

In [25]:
km_anomalies['distance_KM']

Gz9SlLqxS6wnxPvgdOQNrA    3.348413
ZCzey5aPhd7jYIoHsUfjmQ    2.852182
e6d50rwRTU-fONeSBJmOHQ    2.945065
OtGSQQzV7uF8XPAsNIh7Bw    3.130752
I_a74zmgR-X03LsKISWPcg    2.592479
t4P-8drZzj3TMwA_WU-4Zg    2.584595
7SemopLjhDc3IQkI-JUYdQ    2.730502
baIT89GubjGJV1mpn82Eeg    2.851160
1o03kXj4zA-_SvN1HfqNyg    2.774985
Name: distance_KM, dtype: float64

# Gaussian Mixture

In [26]:
##### Gaussian Mixture #########

## choose the number of clusters using silhouette method
get_silhouette_score(X, 'gaussian_mix')

For n_clusters =2,gaussian_mix - average silhouette_score :0.12887906314
For n_clusters =3,gaussian_mix - average silhouette_score :0.119042473774
For n_clusters =4,gaussian_mix - average silhouette_score :0.0681504944473
For n_clusters =5,gaussian_mix - average silhouette_score :0.0553045067472
For n_clusters =6,gaussian_mix - average silhouette_score :0.061464526033
For n_clusters =7,gaussian_mix - average silhouette_score :0.0713501158681
For n_clusters =8,gaussian_mix - average silhouette_score :0.0512525024615
For n_clusters =9,gaussian_mix - average silhouette_score :0.0632702854815


In [27]:
k = 2
GM=GaussianMixture(n_components=k,random_state=22)
GM.fit(X)

label_gm = GM.predict(X)

In [28]:
print list(label_gm).count(0)
print list(label_gm).count(1)
print list(label_gm).count(2)
print list(label_gm).count(3)
print list(label_gm).count(4)

54
122
0
0
0


# Isolation Forest

In [29]:
### 3) Isolation Foreset
from sklearn.ensemble import IsolationForest

# fit the model
ISF = IsolationForest(max_samples=100, random_state=22)
ISF.fit(X)

## compute anomaly score of the input. The lower, the more abnormal.
score_isf = ISF.decision_function(X)

In [30]:
score_isf.argmin()

35

# KNN Distance

In [31]:
def dist2knn(x, nn, k):
    dist_ = []
    for i in range(len(nn)):
        dist_.append(distance.euclidean(x, nn.iloc[i,:]))
    dist_.sort()
    return sum(dist_[:k+1]) # +1: remove 0 self

In [32]:
dist_sum_knn = []
for i in range(len(X)):
    print '\r{}%'.format(100.0*(i+1)/len(X)),
    dist_sum_knn.append(dist2knn(X.iloc[i,:], X, 5))

100.0%


# Output

In [33]:
result = pd.DataFrame(index=X.index)

In [34]:
result['clusters_sp'] = labels_spectural
result['clusters_km'] = labels_km
result['distance_km'] = km_anomalies['distance_KM']
result['clusters_gm'] = label_gm
result['scores_isf'] = score_isf
result['distance_knn'] = dist_sum_knn

In [35]:
result

,clusters_sp,clusters_km,distance_km,clusters_gm,scores_isf,distance_knn
2px99IppAcnxR238eq_8_w,0,0,NaN,1,0.121704,7.706742
cXdQ3H0GqcIzRgrvBEaBxw,0,1,NaN,1,0.073304,8.660254
qMJCVx6-ZYvZQHlfrZ_cug,0,1,NaN,1,0.061064,8.928203
fBmyj1okdO1p-GUzfQQohg,0,1,NaN,1,0.141037,3.000000
umrDQGRNied77aVg29_fVw,0,1,NaN,1,0.017409,12.016867
Y1WIH4jstH846oWcDnoZLw,0,0,NaN,1,0.141601,6.242641
fS9a8AGrcwWPf_9vfn5wPQ,0,0,NaN,0,0.157274,2.000000
uWYzS46xqA_YA1D1INgbEQ,0,0,NaN,0,0.133695,5.828427
aVKBmuvdDmEOgOfqaSF_xA,0,1,NaN,1,0.046627,9.700170
n8KZA9G4FvGqDBWIscpz9Q,0,0,NaN,1,0.152312,5.000000


In [36]:
result.to_csv('Phoenix_chi_results.csv', index_label=False, encoding='utf-8' )